In [ ]:
#pandas根据一列数据更新另一列
def number_to_flag(number):
    if number > 0:
        return '涨'
    elif number == 0:
        return '平'
    else:
        return '跌'

 df['涨跌'] = df['p_change'].map(number_to_flag)


In [ ]:
import numpy as np 
df['涨跌'] = np.sign(df['p_change']) 
df['涨跌'].replace([-1,0,1], ['跌','平','涨'], inplace = True) 

In [ ]:
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt
from django.views.generic.base import View
from lxml import etree
from django.utils.encoding import smart_str
import hashlib
import time
from django.template.loader import render_to_string
from django.views.decorators.csrf import ensure_csrf_cookie

# Create your views here.


#csrf_exempt装饰器是取消Django的csrf标记的，毕竟微信不会有这种标记。这次认证通过之后，这个方法你注销了都行，除非你再次认证，不然不会再使用这个方法了。
@csrf_exempt
#如果request提供，它必须是一个HttpRequest。然后，引擎必须将其以及模板中可用的CSRF令牌。
@ensure_csrf_cookie

def weixin(request):

    if request.method == 'GET':
        #下面这四个参数是在接入时，微信的服务器发送过来的参数
        signature = request.GET.get('signature', None)
        timestamp = request.GET.get('timestamp', None)
        nonce = request.GET.get('nonce', None)
        echostr = request.GET.get('echostr', None)

        #这里的token需要自己设定，主要是和微信的服务器完成验证使用
        token = 'WEIXIN_TOKEN'

        #把token，timestamp, nonce放在一个序列中，并且按字符排序
        hashlist = [token, timestamp, nonce]
        hashlist.sort()

        #将上面的序列合成一个字符串
        hashstr = ''.join([s for s in hashlist])

        #通过python标准库中的sha1加密算法，处理上面的字符串，形成新的字符串。
        hashstr = hashlib.sha1(hashstr.encode(encoding='utf-8')).hexdigest()

        #把我们生成的字符串和微信服务器发送过来的字符串比较，
        #如果相同，就把服务器发过来的echostr字符串返回去
        if hashstr == signature:
          return HttpResponse(echostr)

    if request.method == 'POST':
        #将程序中字符输出到非 Unicode 环境（比如 HTTP 协议数据）时可以使用 smart_str 方法
        data = smart_str(request.body)
        print(data)
        #将接收到数据字符串转成xml
        xml = etree.fromstring(data)

        #从xml中读取我们需要的数据。注意这里使用了from接收的to，使用to接收了from，
        #这是因为一会我们还要用这些数据来返回消息，这样一会使用看起来更符合逻辑关系
        fromUser = xml.find('ToUserName').text
        toUser = xml.find('FromUserName').text
        msgType = xml.find('MsgType').text

        #这里获取当前时间的秒数，time.time()取得的数字是浮点数，所以有了下面的操作
        nowtime = str(int(time.time()))

        #加载text.xml模板,参见render()调用render_to_string()并将结果馈送到 HttpResponse适合从视图返回的快捷方式 。
        if msgType =='text':
            content = xml.find('Content').text
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '文本消息，功能正在开发中'})
            return HttpResponse(rendered)
        if msgType =='image':
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '图片消息，功能正在开发中'})
            return HttpResponse(rendered)
        if msgType =='voice':
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '语音消息，功能正在开发中'})
            return HttpResponse(rendered)
        if msgType =='video':
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '视频消息，功能正在开发中'})
            return HttpResponse(rendered)
        if msgType =='shortvideo':
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '小视频消息，功能正在开发中'})
            return HttpResponse(rendered)
        if msgType =='location':
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '地理位置消息，功能正在开发中'})
            return HttpResponse(rendered)
        if msgType =='link':
            rendered = render_to_string('wechat/text.xml',{'toUser': toUser,'fromUser': fromUser,'nowtime': nowtime,'content': '链接消息，功能正在开发中'})
            return HttpResponse(rendered)

In [ ]:
def TestUrlOpen():
  import urllib.request
  page = urllib.request.urlopen("http://www.webxml.com.cn/WebServices/WeatherWebService.asmx/getWeatherbyCityName?theCityName=58367")
  lines = page.readlines()
  page.close()
  document = ""
  for line in lines :
    document = document + line.decode('utf-8')

  from xml.dom.minidom import parseString
  dom =parseString(document)
  strings = dom.getElementsByTagName("string")
  print (strings[6].childNodes[0].data + " " + strings[7].childNodes[0].data)

In [ ]:
def TestSuds() :
  from suds import WebFault
  from suds.client import Client 
  url = 'http://webservice.webxml.com.cn/WebServices/WeatherWS.asmx?WSDL'
  client = Client(url)
  print (client)
  print (client.service.getWeather('58367'))

In [2]:
500/0.003/10000

16.666666666666664

In [ ]:
Action: ANTITRUST TOMATOES, CLUELESS BUCKET; Animation: CLASH FREDDY, INCH JET, SNOWMAN ROLLERCOASTER; Children: CABIN FLASH, LEGALLY SECRETARY, MODEL FISH, SPLENDOR PATTON; Classics: GALAXY SWEETHEARTS, STEEL SANTA; Comedy: GROUNDHOG UNCUT, LIFE TWISTED, LION UNCUT; Documentary: HOMEWARD CIDER, METAL ARMAGEDDON, WEDDING APOLLO; Drama: TORQUE BOUND; Family: LOLITA WORLD, SPLASH GUMP; Foreign: FRIDA SLIPPER; Games: GRINCH MASSAGE, MOONWALKER FOOL; Horror: MOTIONS DETAILS, ZHIVAGO CORE; Music: ALONE TRIP, REBEL AIRPORT; New: ATTRACTION NEWTON, CHINATOWN GLADIATOR; Sci-Fi: DAISY MENAGERIE, SOLDIERS EVOLUTION; Sports: DRIVER ANNIE, RIDER CADDYSHACK; Travel: BOONDOCK BALLROOM, LEATHERNECKS DWARFS

In [2]:
actor_info = 'Action: ANTITRUST TOMATOES, CLUELESS BUCKET; Animation: CLASH FREDDY, INCH JET, SNOWMAN ROLLERCOASTER; Children: CABIN FLASH, LEGALLY SECRETARY, MODEL FISH, SPLENDOR PATTON; Classics: GALAXY SWEETHEARTS, STEEL SANTA; Comedy: GROUNDHOG UNCUT, LIFE TWISTED, LION UNCUT; Documentary: HOMEWARD CIDER, METAL ARMAGEDDON, WEDDING APOLLO; Drama: TORQUE BOUND; Family: LOLITA WORLD, SPLASH GUMP; Foreign: FRIDA SLIPPER; Games: GRINCH MASSAGE, MOONWALKER FOOL; Horror: MOTIONS DETAILS, ZHIVAGO CORE; Music: ALONE TRIP, REBEL AIRPORT; New: ATTRACTION NEWTON, CHINATOWN GLADIATOR; Sci-Fi: DAISY MENAGERIE, SOLDIERS EVOLUTION; Sports: DRIVER ANNIE, RIDER CADDYSHACK; Travel: BOONDOCK BALLROOM, LEATHERNECKS DWARFS';

In [3]:
actor_info

'Action: ANTITRUST TOMATOES, CLUELESS BUCKET; Animation: CLASH FREDDY, INCH JET, SNOWMAN ROLLERCOASTER; Children: CABIN FLASH, LEGALLY SECRETARY, MODEL FISH, SPLENDOR PATTON; Classics: GALAXY SWEETHEARTS, STEEL SANTA; Comedy: GROUNDHOG UNCUT, LIFE TWISTED, LION UNCUT; Documentary: HOMEWARD CIDER, METAL ARMAGEDDON, WEDDING APOLLO; Drama: TORQUE BOUND; Family: LOLITA WORLD, SPLASH GUMP; Foreign: FRIDA SLIPPER; Games: GRINCH MASSAGE, MOONWALKER FOOL; Horror: MOTIONS DETAILS, ZHIVAGO CORE; Music: ALONE TRIP, REBEL AIRPORT; New: ATTRACTION NEWTON, CHINATOWN GLADIATOR; Sci-Fi: DAISY MENAGERIE, SOLDIERS EVOLUTION; Sports: DRIVER ANNIE, RIDER CADDYSHACK; Travel: BOONDOCK BALLROOM, LEATHERNECKS DWARFS'

In [4]:
actor_info.split(';')

['Action: ANTITRUST TOMATOES, CLUELESS BUCKET',
 ' Animation: CLASH FREDDY, INCH JET, SNOWMAN ROLLERCOASTER',
 ' Children: CABIN FLASH, LEGALLY SECRETARY, MODEL FISH, SPLENDOR PATTON',
 ' Classics: GALAXY SWEETHEARTS, STEEL SANTA',
 ' Comedy: GROUNDHOG UNCUT, LIFE TWISTED, LION UNCUT',
 ' Documentary: HOMEWARD CIDER, METAL ARMAGEDDON, WEDDING APOLLO',
 ' Drama: TORQUE BOUND',
 ' Family: LOLITA WORLD, SPLASH GUMP',
 ' Foreign: FRIDA SLIPPER',
 ' Games: GRINCH MASSAGE, MOONWALKER FOOL',
 ' Horror: MOTIONS DETAILS, ZHIVAGO CORE',
 ' Music: ALONE TRIP, REBEL AIRPORT',
 ' New: ATTRACTION NEWTON, CHINATOWN GLADIATOR',
 ' Sci-Fi: DAISY MENAGERIE, SOLDIERS EVOLUTION',
 ' Sports: DRIVER ANNIE, RIDER CADDYSHACK',
 ' Travel: BOONDOCK BALLROOM, LEATHERNECKS DWARFS']

In [5]:
for pair in actor_info.split(';'):
    key, value = pair.split(':')
    print('Actor_info[{}] = {}'.format(key, value))

Actor_info[Action] =  ANTITRUST TOMATOES, CLUELESS BUCKET
Actor_info[ Animation] =  CLASH FREDDY, INCH JET, SNOWMAN ROLLERCOASTER
Actor_info[ Children] =  CABIN FLASH, LEGALLY SECRETARY, MODEL FISH, SPLENDOR PATTON
Actor_info[ Classics] =  GALAXY SWEETHEARTS, STEEL SANTA
Actor_info[ Comedy] =  GROUNDHOG UNCUT, LIFE TWISTED, LION UNCUT
Actor_info[ Documentary] =  HOMEWARD CIDER, METAL ARMAGEDDON, WEDDING APOLLO
Actor_info[ Drama] =  TORQUE BOUND
Actor_info[ Family] =  LOLITA WORLD, SPLASH GUMP
Actor_info[ Foreign] =  FRIDA SLIPPER
Actor_info[ Games] =  GRINCH MASSAGE, MOONWALKER FOOL
Actor_info[ Horror] =  MOTIONS DETAILS, ZHIVAGO CORE
Actor_info[ Music] =  ALONE TRIP, REBEL AIRPORT
Actor_info[ New] =  ATTRACTION NEWTON, CHINATOWN GLADIATOR
Actor_info[ Sci-Fi] =  DAISY MENAGERIE, SOLDIERS EVOLUTION
Actor_info[ Sports] =  DRIVER ANNIE, RIDER CADDYSHACK
Actor_info[ Travel] =  BOONDOCK BALLROOM, LEATHERNECKS DWARFS


In [15]:
d_actor_info = {key.strip(): [v.strip() for v in value.split(',')] for key, value in [ai.split(':') for ai in actor_info.split(';')]}

In [16]:
d_actor_info

{'Action': ['ANTITRUST TOMATOES', 'CLUELESS BUCKET'],
 'Animation': ['CLASH FREDDY', 'INCH JET', 'SNOWMAN ROLLERCOASTER'],
 'Children': ['CABIN FLASH',
  'LEGALLY SECRETARY',
  'MODEL FISH',
  'SPLENDOR PATTON'],
 'Classics': ['GALAXY SWEETHEARTS', 'STEEL SANTA'],
 'Comedy': ['GROUNDHOG UNCUT', 'LIFE TWISTED', 'LION UNCUT'],
 'Documentary': ['HOMEWARD CIDER', 'METAL ARMAGEDDON', 'WEDDING APOLLO'],
 'Drama': ['TORQUE BOUND'],
 'Family': ['LOLITA WORLD', 'SPLASH GUMP'],
 'Foreign': ['FRIDA SLIPPER'],
 'Games': ['GRINCH MASSAGE', 'MOONWALKER FOOL'],
 'Horror': ['MOTIONS DETAILS', 'ZHIVAGO CORE'],
 'Music': ['ALONE TRIP', 'REBEL AIRPORT'],
 'New': ['ATTRACTION NEWTON', 'CHINATOWN GLADIATOR'],
 'Sci-Fi': ['DAISY MENAGERIE', 'SOLDIERS EVOLUTION'],
 'Sports': ['DRIVER ANNIE', 'RIDER CADDYSHACK'],
 'Travel': ['BOONDOCK BALLROOM', 'LEATHERNECKS DWARFS']}